# Φόρτωση Δεδομένων Εικόνας

Μέχρι στιγμής έχουμε δει απλά και τεχνητά σύνολα δεδομένων εκπαίδευσης που συνήθως δεν θα χρησιμοποιούσαμε σε πραγματικά προβλήματα. Αντίθετως, το πιο πιθανό ειναι να ασχοληθείτε με εικόνες πλήρους μεγέθους, όπως π.χ εικόνες από τα κινητά σας τηλέφωνα. Σε αυτό το notebook, θα δούμε πώς να φορτώνουμε εικόνες και να τις χρησιμοποιούμε για την εκπαίδευση νευρωνικών δικτύων.

Θα χρησιμοποιήσουμε ένα [dataset απο εικόνες σκύλων και γατών](https://www.kaggle.com/c/dogs-vs-cats) το οποίο ειναι διαθέσιμο στο Kaggle. Παρακάτω εμφανίζονται δύο εικόνες απο το dataset:

<img src='assets/dog_cat.png'>

Θα χρησιμοποιήσουμε αυτό το σύνολο δεδομένων για να εκπαιδεύσουμε ένα νευρωνικό δίκτυο για να κάνει διάκριση μεταξύ γατών και σκύλων. Σήμερα η δημιουργία ενός τέτοιου δικτύου δεν φαίνεται μεγάλο επίτευγμα, αλλά πριν από πέντε χρόνια ήταν μια σοβαρή πρόκληση για τα συστήματα όρασης υπολογιστών.

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

import torch
from torchvision import datasets, transforms

import helper

Ο πιο εύκολος τρόπος για να φορτώσουμε τις εικόνες μας ειναι με το `datasets.ImageFolder` απο τη `torchvision` ([documentation](http://pytorch.org/docs/master/torchvision/datasets.html#imagefolder)). Γενικά, θα χρησιμοποιείται το `ImageFolder` όπως παρακάτω:

```python
dataset = datasets.ImageFolder('path/to/data', transform=transform)
```

όπου `'path/to/data'` είναι ο φάκελος στον οποίο περιέχονται οι εικόνες σας και `transform` είναι μία αλληλουχεία βημάτων επεξεργασίας βασισμένα στο module [`transforms`](http://pytorch.org/docs/master/torchvision/transforms.html) απο τη `torchvision`. Το ImageFolder περιμένει τα αρχεία και οι φάκελοί να έχουν την παρακάτω δομή:
```
root/dog/xxx.png
root/dog/xxy.png
root/dog/xxz.png

root/cat/123.png
root/cat/nsdf3.png
root/cat/asd932_.png
```

όπου η κάθε κλάση έχει το δικό της directory (`cat` και `dog`) για τις εικόνες. Οι εικόνες στη συνέχεια αυτόματα παίρνουν την ετικέτα τους (label) απο το όνομα του φακέλου. Στο παράδειγμά μας, η εικόνα `123.png` θα φορτωθεί με όνομα κλάσης `cat`. Μπορείτε να κατεβάσετε όλο το dataset με τη παραπάνω δομή [απο εδώ](https://s3.amazonaws.com/content.udacity-data.com/nd089/Cat_Dog_data.zip). Τα δεδομένα είναι επίσης ήδη διαχωρισμένα σε σετ εκπαίδευσης και ελέγχου.

### Μετασχηματισμοί (Transforms)

Όταν φορτώνετε τα δεδομένα σας με το `ImageFolder`, θα πρέπει να καθωρίσετε και καποιους μετασχηματισμούς. Για παράδειγμα, οι εικόνες έχουν διαφορετικά μεγέθη, αλλά χρειαζόμαστε όλες τις εικόνες να έχουν το ίδιο μέγεθος για την εκπαίδευση. Μπορείτε να αλλάξετε το μέγεθός τους είτε με το `transforms.Resize()` ή να τις κάνετε crop με την `transforms.CenterCrop()`, `transforms.RandomResizedCrop()`, κτλ. Πρέπει επίσης να μετατρέψουμε τις εικόνες μας σε τανυστές PyTorch μέσω της `transforms.ToTensor()`. Συνήθως συνδιάζουμε όλα τα βήματα των μετασχηματισμών σε μία αλληλουχία βημάτων με την `transforms.Compose()`, η οποία θα δέχεται μία λίστα απο μετασχηματισμούς και τους εκτελεί στη συνέχεια με τη σειρά που τους δηλώσαμε. Κάπως έτσι για παράδειγμα ειναι μία αλληλουχία βημάτων για κλιμάκωση, crop και μετατροπή σε τανυστές:

```python
transform = transforms.Compose([transforms.Resize(255),
                                transforms.CenterCrop(224),
                                transforms.ToTensor()])

```

Υπάρχουν πολλοί διαθέσιμοι μετασχηματισμοί, θα καλύψουμε μερικούς και αργότερα αλλά μπορείτε να διαβάσετε και εδω [documentation](http://pytorch.org/docs/master/torchvision/transforms.html). 

### Data Loaders

Με την `ImageFolder` να έχει εκτελεστεί, πρέπει να περάσετε σαν όρισμα το dataset στον [`DataLoader`](http://pytorch.org/docs/master/data.html#torch.utils.data.DataLoader). Ο `DataLoader` δέχετε το dataset (όπως ακριβώς θα κάνατε και με τον `ImageFolder`) και επιστρέγει παρτίδες εικόνων και τις αντίστοιχες ετικέτες τους. Μπορείτε να ορίσετε διάδορες παραμέτρους όπως το μέγεθος της παρτίδας και εαν θέλετε να δεδομένα να ανακατεύονται τυχαία μετά απο κάθε εποχή.

```python
dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)
```

Εδώ ο `dataloader` είναι ένας [generator](https://jeffknupp.com/blog/2013/04/07/improve-your-python-yield-and-generators-explained/). Για να τραβήξετε τα δεδομένα του, θα πρέπει να τον εκτελέσετε σε ένα βρόχο ή να τον μετατρέψετε σε έναν iterator και να εκτελέσετε την εντολη `next()`.

```python
# Σε βρόχο, παίρνω μία παρτίδα σε κάθε επανάληψη 
for images, labels in dataloader:
    pass

# Πάρτε μία παρτίδα
images, labels = next(iter(dataloader))
```
 
>**Άσκηση:** Φορτώστε τις εικόνες απο το ΄φακελο `Cat_Dog_data/train`, ορίστε μερικούς μετασχηματισμούς, και στη συνέχεια δομήστε τον dataloader.

In [ ]:
data_dir = 'Cat_Dog_data/train'

transform = transforms.Compose([transforms.Resize(255),
                                transforms.CenterCrop(224),
                                transforms.ToTensor()])
dataset = datasets.ImageFolder(data_dir, transform=transform)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)

In [ ]:
# Εκτελέστε αυτο το μπλοκ, για να ελεγξετε τον data loader σας
images, labels = next(iter(dataloader))
helper.imshow(images[0], normalize=False)

Αν φορτώσατε τα δεδομένα σας σωστά, θα πρέπει να δείτε κάτι τέτοιο (η εικόνα σας θα ειναι διαφορετική):

<img src='assets/cat_cropped.png' width=244>

## Αύξηση Δεδομένων Εκπαιδευσης (Data Augmentation)

Μια κοινή στρατηγική για την εκπαίδευση νευρωνικών δικτύων είναι η εισαγωγή τυχαιότητας στα ίδια τα δεδομένα εισόδου. Για παράδειγμα, μπορείτε να περιστρέψετε, να αναστρέψετε (mirror), να κλιμακώσετε ή / και να περικόψετε (crop) τις εικόνες σας τυχαία κατά τη διάρκεια της εκπαίδευσης. Αυτό θα βοηθήσει το δίκτυό σας να γενικεύσει καθώς βλέπει τις ίδιες εικόνες αλλά σε διαφορετικές θέσεις, με διαφορετικά μεγέθη, με διαφορετικούς προσανατολισμούς κ.λπ.

Για ένα τυχαίο rotate, scale και crop, και μετά flip των εικόνων θα πρέπει να ορίσετε τους μετασχηματισμούς όπως εξής:

```python
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.5, 0.5, 0.5], 
                                                            [0.5, 0.5, 0.5])])
```
Συνήθως, θα θέλετε επίσης να κανονικοποιήσετε τις εικόνες με το `transforms.Normalize`. Αν περάσετε σε μια λίστα τις μέσες τιμές και μια λίστα με τις τυπικές αποκλίσεις, τότε τα κανάλια χρώματος κανονικοποιούνται όπως παρακατω

```input[channel] = (input[channel] - mean[channel]) / std[channel]```

Η αφαίρεση του `mean` κεντράρει τα δεδομένα γύρω από το μηδέν και διαιρώντας με το `std`, οι τιμές κυμαίνονται μεταξύ -1 και 1. Η κανονικοποίηση βοηθά στη διατήρηση των βαρών του δικτύου κοντά στο μηδέν, γεγονός που με τη σειρά του καθιστά το backpropagation πιο σταθερό. Χωρίς κανονικοποίηση, τα δίκτυα θα τείνουν να αποτύχουν να μάθουν.

Η πληρης λίστα των μετασχηματισμών είναι εδω [the available transforms here](http://pytorch.org/docs/0.3.0/torchvision/transforms.html). Ωστόσο, όταν ελεγχετε το δικτυό σας, θα θελήσετε να χρησιμοποιήσετε εικόνες που δεν έχουν τροποποιηθεί πέρα απο τη κανονικοποίηση. Έτσι, για την επικύρωση / έλεγχο, συνήθως θα αλλάξετε μονο το μέγεθος και θα περικόψετε τις εικόνες.

>**Άσκηση:** Ορίστε μετασχηματισμούς για τα δεδομένα εκπαίδευσης και ελέγχου παρακάτω. Αγνοήστε προς το παρον την κανονικοποίησή τους.

In [ ]:
data_dir = 'Cat_Dog_data'

# Συπμληρώστε: Ορίστε μετασχηματισμούς για τα δεδομένα εκπαίδευσης και ελέγχου
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor()]) 

test_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor()])


# Περάστε τους μετασχηματισμούς εδώ, στη συνέχεια, εκτελέστε το επόμενο κελί για να δείτε πώς φαίνονται οι μετασχηματισμοί
train_data = datasets.ImageFolder(data_dir + '/train', transform=train_transforms)
test_data = datasets.ImageFolder(data_dir + '/test', transform=test_transforms)

trainloader = torch.utils.data.DataLoader(train_data, batch_size=32)
testloader = torch.utils.data.DataLoader(test_data, batch_size=32)

In [ ]:
# αλ΄λάξτε το μεταξύ trainloader ή testloader 
data_iter = iter(trainloader)

images, labels = next(data_iter)
fig, axes = plt.subplots(figsize=(10,4), ncols=4)
for ii in range(4):
    ax = axes[ii]
    helper.imshow(images[ii], ax=ax, normalize=False)

Οι μετασχηματισμένες εικόνες σας πρέπει να μοιάζουν κάπως έτσι.

<center>Training examples:</center>
<img src='assets/train_examples.png' width=500px>

<center>Testing examples:</center>
<img src='assets/test_examples.png' width=500px>

Σε αυτό το σημείο θα πρέπει να μπορείτε να φορτώνετε τα δεδομένα για εκπαίδευση και τον έλεγχο. Τώρα, πρέπει να δοκιμάσετε να δημιουργήσετε ένα δίκτυο που να μπορεί να ταξινομήσει τις γάτες έναντι των σκύλων. Αυτό είναι λίγο πιο περίπλοκο από πριν σε σχέση με τα σύνολα δεδομένων MNIST και Fashion-MNIST. Για να είμαι ειλικρινής, πιθανότατα δεν θα το κάνετε να λειτουργήσει σωστά με ένα πλήρως συνδεδεμένο δίκτυο, ανεξάρτητα από το πόσο βαθή είναι. Αυτές οι εικόνες έχουν τρία κανάλια χρώματος και σε υψηλότερη ανάλυση (μέχρι στιγμής έχετε δει εικόνες 28x28 που είναι μικροσκοπικές).

Στο επόμενο μέρος, θα σας δείξω πώς να χρησιμοποιήσετε ένα προ-εκπαιδευμένο δίκτυο για να δημιουργήσετε ένα μοντέλο που μπορεί πραγματικά να λύσει αυτό το πρόβλημα.

In [ ]:
# Προαιρετικό: Προσπαθήστε να δημιουργήσετε ένα δίκτυο για την ταξινόμηση γατών έναντι σκύλων από αυτό το σύνολο δεδομένων

![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)

Αυτό το notebook 📖 δημιουργήθηκε για το μάθημα ***Υπολογιστική Νοημοσύνη και Μηχανική Μάθηση*** του Τμήματος Μηχανικών Παραγωγής και Διοίκησης, της Πολυτεχνικής Σχολής του Δημοκριτείου Πανεπιστημίου Θράκης.<br>
This notebook is made available under the Creative Commons Attribution [(CC-BY)](https://creativecommons.org/licenses/by/4.0/legalcode) license. Code is also made available under the [MIT License](https://opensource.org/licenses/MIT).<br>
Author: Asst. Prof. Angelos Amanatiadis
<img src="assets/cc.png" style="width:55px; float: right; margin: 0px 0px 0px 0px;"></img>
<img src="assets/mit.png" style="width:40px; float: right; margin: 0px 10px 0px 0px;"></img>